## 1. Data Preparation

In [21]:
# add answers data
# Import Modules
import pandas as pd
import numpy as np
import sqlite3
import pickle
import os
import matplotlib.pyplot as plt

# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date, owner_user_id, body, parent_id
FROM answers
WHERE creation_date > '2021-09-01'
AND creation_date < '2023-09-01';
'''
ans_df = pd.read_sql_query(query, conn)
conn.close()

# Import Users Data
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date
FROM users;
'''
user_df = pd.read_sql_query(query, conn)
conn.close()
user_df = user_df.rename(columns = {'creation_date':'account_creation_date',
                                    'id': 'owner_user_id'})
ans_df = ans_df.dropna(subset = 'owner_user_id')
ans_df['owner_user_id'] = ans_df['owner_user_id'].astype(int).astype(str)
# Merge answers to the correspond users
user_df['owner_user_id'] = user_df['owner_user_id'].astype(str)
ans_df = pd.merge(ans_df, user_df, on = 'owner_user_id', how = 'left')
ans_df

,id,creation_date,owner_user_id,body,parent_id,account_creation_date
0,69006420,2021-09-01 00:00:18.070,7332046,"<p>The short approach, remove python3.9 from y...",69006229,2016-12-22 19:33:51.100
1,69006421,2021-09-01 00:00:28.823,14471093,"<p>open the terminal, run as administrator.</p...",68746577,2020-10-18 06:29:51.557
2,69006422,2021-09-01 00:00:35.537,523612,"<p>Conceptually, <code>Base</code> has no reas...",69006293,2010-11-29 07:44:17.673
3,69006429,2021-09-01 00:01:09.140,11269158,<p><code>calldata</code> is a special data loc...,68997666,2019-03-27 23:15:22.047
4,69006430,2021-09-01 00:01:17.920,2096113,<p>You could do something like this if the str...,69006320,2013-02-21 15:43:28.953
...,...,...,...,...,...,...
3102831,77019850,2023-08-31 23:56:07.187,313768,<p>Make an inner solution routine that accepts...,77013377,2010-04-11 04:47:56.083
3102832,77019851,2023-08-31 23:56:19.123,13296852,<p>When you pass <code>stack[100]</code> as an...,77019825,2020-04-12 21:26:44.880
3102833,77019853,2023-08-31 23:56:47.647,5103949,"<p>I had the same error, I fixed it by followi...",76922631,2015-07-10 17:53:49.643
3102834,77019855,2023-08-31 23:57:36.797,16844882,"<p>In your Razor component, you can inject the...",77019829,2021-09-06 16:29:07.503


### 1-1 Divide Dataset into two parts (OB and NB)

In [22]:
# Compute difference between post creation_date - user creation_date.
ans_df['month_diff'] = (pd.to_datetime(ans_df['creation_date']) - pd.to_datetime(ans_df['account_creation_date'])) / pd.Timedelta(days=30)
ans_df = ans_df[ans_df['month_diff'] >= 0]
ans_df['year_month_day'] = pd.to_datetime(ans_df['creation_date']).dt.to_period('D').astype(str)
ans_df['account_year_month'] = pd.to_datetime(ans_df['account_creation_date']).dt.to_period('M')
df_new_a = ans_df[ans_df['account_year_month'] >= '2020-09-01']
df_old_a = ans_df[ans_df['account_year_month'] < '2020-09-01']

/tmp/ipykernel_1361730/1871056578.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans_df['year_month_day'] = pd.to_datetime(ans_df['creation_date']).dt.to_period('D').astype(str)
/tmp/ipykernel_1361730/1871056578.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans_df['account_year_month'] = pd.to_datetime(ans_df['account_creation_date']).dt.to_period('M')


### 1-2 Filter Users who Constantly Wrote Posts

In [23]:
def filterUsers(data):
    # erase NAs
    data = data.dropna(subset=['owner_user_id'])
    # Convert types
    data['owner_user_id'] = data['owner_user_id'].astype(int).astype(str)
    data['creation_date'] = pd.to_datetime(data['creation_date'])

    # add year_month variable
    data['year_month'] = data['creation_date'].dt.to_period('M')

    # Get the total number of unique months
    total_months = data['year_month'].nunique()
    # Count the number of unique months for each user
    user_month_counts = data.groupby('owner_user_id')['year_month'].nunique()
    # Filter users who have written posts in every single month
    users_in_every_month = user_month_counts[user_month_counts == total_months].index.tolist()

    # Filtered data
    df_filtered = data[data['owner_user_id'].isin(users_in_every_month)]
    return df_filtered

In [24]:
filtered_old = filterUsers(df_old_a)
filtered_new = filterUsers(df_new_a)

In [25]:
print("Number of Users in old dataset: ", filtered_old.owner_user_id.nunique())
print("Number of Users in new dataset: ", filtered_new.owner_user_id.nunique())

Number of Users in old dataset:  769
Number of Users in new dataset:  26


### 1-3 Get Quetions' tag data and attach.

In [26]:
# Import Questions data
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, tags
FROM questions
WHERE creation_date > '2020-09-01'
AND creation_date < '2023-09-01';
'''
df_tags = pd.read_sql_query(query, conn)
conn.close()

In [27]:
# For old
df_tags['id'] = df_tags['id'].astype(str)
filtered_old['parent_id'] = filtered_old['parent_id'].astype(int).astype(str)
df_merge_old = pd.merge(filtered_old, df_tags, left_on = 'parent_id', right_on = 'id', how = 'left')
df_merge_old.drop('id_y', axis = 1, inplace = True)
df_merge_old = df_merge_old.rename(columns = {'id_x': 'id'})

# For new
filtered_new['parent_id'] = filtered_new['parent_id'].astype(int).astype(str)
df_merge_new = pd.merge(filtered_new, df_tags, left_on = 'parent_id', right_on = 'id', how = 'left')
df_merge_new.drop('id_y', axis = 1, inplace = True)
df_merge_new = df_merge_new.rename(columns = {'id_x': 'id'})

### 1-4 Aggregate tags of each user

In [29]:
def aggTags(data):
    data['tags'] = data['tags'].str.replace('[\["\]]', '', regex=True)
    data['tags'] = data['tags'].str.replace('c#','Csharp')
    data['tags'] = data['tags'].str.replace('c++','Cpp')
    data['tags'] = data['tags'].str.replace('.','DOT')
    data['tags'] = data['tags'].str.replace('><',' ')
    data['tags'] = data['tags'].str.replace('>','')
    data['tags'] = data['tags'].str.replace('-','')
    data['tags'] = data['tags'].str.replace('"', '')
    new_merge = data.groupby('owner_user_id')[['tags']].agg(lambda x: ', '.join(map(str,x))).reset_index()
    return data, new_merge

In [30]:
df_merge_old, new_merge_old = aggTags(df_merge_old)
df_merge_new, new_merge_new = aggTags(df_merge_new)

In [31]:
new_merge_old

,owner_user_id,tags
0,10008173,"docker, filesystems, docker, express, dockerfi..."
1,10112124,"angular, authentication, rxjs, ngrx, ngrxentit..."
2,10138734,"mysql, rubyonrails, ruby, railsmigrations, mys..."
3,10140124,"elasticsearch, javascript, ifstatement, boolea..."
4,10157127,"flutter, flutterlayout, flutterdependencies, f..."
...,...,...
764,9952196,"sqlite, linux, bash, shell, sed, linux, bash, ..."
765,9957710,"arrays, struct, julia, arrays, julia, multithr..."
766,99692,"rust, rust, enums, io, rust, rust, lifetime, o..."
767,9971759,"python, ortools, python, constraints, nonlinea..."


### 1-5 Find the majority of tags and assign a proper cluster.

In [32]:
new_merge_old['majority_tag'] = new_merge_old['tags'].apply(lambda x: x.split(', ')[0] if len(set(x.split(', '))) == 1 else max(set(x.split(', ')), key = x.split(', ').count))
new_merge_new['majority_tag'] = new_merge_new['tags'].apply(lambda x: x.split(', ')[0] if len(set(x.split(', '))) == 1 else max(set(x.split(', ')), key = x.split(', ').count))
new_merge_new

,owner_user_id,tags,majority_tag
0,14460824,"python, beautifulsoup, python, beautifulsoup, ...",python
1,14536215,"python, fastapi, pydantic, python, anaconda, c...",python
2,14574199,"aspDOTnetcore, aspDOTnetwebapi, azureactivedir...",aspDOTnetcore
3,14624409,"python, airflow, directedacyclicgraphs, postgr...",airflow
4,14672114,"awk, javascript, arrays, ifstatement, repeat, ...",awk
5,14704799,"amazonwebservices, kubernetes, amazonvpc, cidr...",kubernetes
6,14726230,"flutter, dart, flutterlayout, android, flutter...",flutter
7,14732669,"azuredatafactory, mongodb, mongoose, mongodbqu...",mongodb
8,14853083,"php, arrays, explode, mysql, python, php, arra...",php
9,14868997,"sqlserver, sql, xml, tsql, xquery, xquerysql, ...",sqlserver


In [33]:
# Pre computed clusters
import pickle
with open(file = '/data1/StackOverflow/_Robustness/TagCluster/louvain_community_pre.pickle', mode = 'rb') as file:
    df_clusters = pickle.load(file)

In [37]:
# (OldBie)
# merge with df_clusters
df_post_cluster_old = pd.merge(new_merge_old, df_clusters, left_on = 'majority_tag', right_on = 'tag', how = 'left')
# preProcess (erase columns = tag, tag_count / community feature as an integer)
df_post_cluster_old = df_post_cluster_old.drop(['tag', 'tag_count'], axis = 1)
df_post_cluster_old.groupby('community').size()

community
0     102
1      79
3      65
4     128
5      49
6      83
7      39
8      25
9       2
10      3
11      5
13     13
14      2
15      3
16      1
17    170
dtype: int64

In [38]:
# (NewBie)
# merge with df_clusters
df_post_cluster_new = pd.merge(new_merge_new, df_clusters, left_on = 'majority_tag', right_on = 'tag', how = 'left')
# preProcess (erase columns = tag, tag_count / community feature as an integer)
df_post_cluster_new = df_post_cluster_new.drop(['tag', 'tag_count'], axis = 1)
df_post_cluster_new.groupby('community').size()

community
0     3
1     1
3     1
4     3
5     1
6     5
7     3
8     2
13    2
17    5
dtype: int64

## 2. LSM

In [39]:
# (Oldbie)
df_analysis_old = df_merge_old.groupby(['owner_user_id', 'year_month'])['body'].agg(lambda x: ', '.join(map(str,x))).reset_index()
df_analysis_old = pd.merge(df_analysis_old, df_post_cluster_old.drop(['tags', 'majority_tag'], axis = 1), on = 'owner_user_id', how = 'left')
# remove 16th community
df_analysis_old = df_analysis_old[df_analysis_old['community'] != 16]
df_analysis_old

,owner_user_id,year_month,body,community
0,10008173,2021-09,<p>Since you already know the location on the ...,7
1,10008173,2021-10,<p>That <code>ENTRYPOINT</code> line doesn't f...,7
2,10008173,2021-11,<p>A Docker container only runs one process. ...,7
3,10008173,2021-12,<p>Connections between containers (over the Do...,7
4,10008173,2022-01,<p>You can use the Docker image name (<code>do...,7
...,...,...,...,...
18451,997358,2023-04,<p>To avoid various inefficiencies in your pro...,4
18452,997358,2023-05,<p>If you want an expression which is guarante...,4
18453,997358,2023-06,<p>How to optimize? Since you don't seem to h...,4
18454,997358,2023-07,"<p>Since the <a href=""/questions/tagged/jq"" cl...",4


In [40]:
# size of community = # of users in a cluster x 24
df_analysis_old.groupby('community').size()

community
0     2448
1     1896
3     1560
4     3072
5     1176
6     1992
7      936
8      600
9       48
10      72
11     120
13     312
14      48
15      72
17    4080
dtype: int64

In [49]:
# (Newbie)
df_analysis_new = df_merge_new.groupby(['owner_user_id', 'year_month'])['body'].agg(lambda x: ', '.join(map(str,x))).reset_index()
df_analysis_new = pd.merge(df_analysis_new, df_post_cluster_new.drop(['tags', 'majority_tag'], axis = 1), on = 'owner_user_id', how = 'left')
# remove 1, 3, 5 community
df_analysis_new = df_analysis_new[(df_analysis_new['community'] != 1)&(df_analysis_new['community'] != 3)&(df_analysis_new['community'] != 5)]
df_analysis_new

,owner_user_id,year_month,body,community
0,14460824,2021-09,<p>To count / print only the headings from the...,17
1,14460824,2021-10,<h3>solution bs4 only</h3>\n<p>Select all <cod...,17
2,14460824,2021-11,"<p>It is not necessary to use all these loops,...",17
3,14460824,2021-12,<h3>How to (quick) fix with <code>beautifulsou...,17
4,14460824,2022-01,<p>As mentioned expected result is not that cl...,17
...,...,...,...,...
619,17026934,2023-04,<p>This is a bug in <code>iterate</code> which...,13
620,17026934,2023-05,"<p>Tim Bradshaw's <a href=""https://tfeb.github...",13
621,17026934,2023-06,"<p>It uses <a href=""https://edicl.github.io/hu...",13
622,17026934,2023-07,<p>To elaborate on the other answer. If you s...,13


In [50]:
# size of community = # of users in a cluster x 24
df_analysis_new.groupby('community').size()

community
0      72
4      72
6     120
7      72
8      48
13     48
17    120
dtype: int64

In [54]:
# save data
with open(f'df_analysis_old.pickle', 'wb') as fw:
    pickle.dump(df_analysis_old, fw)
with open(f'df_analysis_new.pickle', 'wb') as fw:
    pickle.dump(df_analysis_new, fw)

### 2-1 Make aggregated MD file for OB and NB

In [52]:
# Save individual body data into separate md file.
year_month = list(df_analysis_old['year_month'].unique().astype('str')) # get unique year_month
for i in range(len(year_month)):
    output_directory = f'/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/oldbie/{year_month[i]}'
    os.makedirs(output_directory, exist_ok=True)
    data = df_analysis_old[df_analysis_old['year_month'] == year_month[i]]
    for index, row in data.iterrows():
        user_filename = f"{output_directory}/user_{row['owner_user_id']}.md"
        with open(user_filename, 'w') as md_file:
            md_file.write(f"## User {row['owner_user_id']}\n")
            md_file.write(row['body'] + '\n\n')
    print(f"Data has been saved to individual md files in the '{output_directory}' directory.")

Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/oldbie/2021-09' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/oldbie/2021-10' directory.


Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/oldbie/2021-11' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/oldbie/2021-12' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/oldbie/2022-01' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/oldbie/2022-02' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/oldbie/2022-03' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/oldbie/2022-04' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/oldbie/2022-05' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness

In [53]:
# Save individual body data into separate md file.
year_month = list(df_analysis_new['year_month'].unique().astype('str')) # get unique year_month
for i in range(len(year_month)):
    output_directory = f'/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/newbie/{year_month[i]}'
    os.makedirs(output_directory, exist_ok=True)
    data = df_analysis_new[df_analysis_new['year_month'] == year_month[i]]
    for index, row in data.iterrows():
        user_filename = f"{output_directory}/user_{row['owner_user_id']}.md"
        with open(user_filename, 'w') as md_file:
            md_file.write(f"## User {row['owner_user_id']}\n")
            md_file.write(row['body'] + '\n\n')
    print(f"Data has been saved to individual md files in the '{output_directory}' directory.")

Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/newbie/2021-09' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/newbie/2021-10' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/newbie/2021-11' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/newbie/2021-12' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/newbie/2022-01' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/newbie/2022-02' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_data/newbie/2022-03' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness

### 2-2 Use separate py file for LSM Computation
- compute.py

#### 2-2-1 Newbie

In [56]:
with open('df_analysis_new.pickle', 'rb') as fw:
    df_analysis_new = pickle.load(fw)
print("Number of observation: ", len(df_analysis_new))
print("NumUser x Num year_months: ", df_analysis_new.owner_user_id.nunique()*df_analysis_new.year_month.nunique())

Number of observation:  552
NumUser x Num year_months:  552


In [57]:
# LSM Score Aggregation
import os
import pandas as pd
folder_path = '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_result/newbie'
# Get a list of all files in the folder
file_list = os.listdir(folder_path)
file_list = sorted(file_list)
# Filter only pickle files
pickle_files = [file for file in file_list if file.endswith('.pickle')]

# Iterate through each pickle file and load it into a separate object
loaded_objects = pd.DataFrame(columns = ['User1', 'year_month', 'Similarity_toAvg'])
for pickle_file in pickle_files:
    file_path = os.path.join(folder_path, pickle_file)
    file_name = os.path.splitext(pickle_file)[0]
    # AGGREGATION
    with open(file_path, 'rb') as f:
        loaded_object = pickle.load(f)
    loaded_object['Similarity_toAvg'] = loaded_object['Similarity_toAvg'].astype(float)
    loaded_object = loaded_object.groupby('User1')['Similarity_toAvg'].mean().reset_index()
    loaded_object['year_month'] = file_name # Add year_month
    loaded_object = loaded_object[['User1', 'year_month', 'Similarity_toAvg']]
    # APPEND
    loaded_objects = pd.concat([loaded_objects, loaded_object])

/tmp/ipykernel_1361730/1205205983.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  loaded_objects = pd.concat([loaded_objects, loaded_object])


In [58]:
loaded_objects

,User1,year_month,Similarity_toAvg
0,user_14460824.md,2021-09,0.170243
1,user_14536215.md,2021-09,0.162675
2,user_14624409.md,2021-09,0.175510
3,user_14672114.md,2021-09,0.162087
4,user_14704799.md,2021-09,0.171497
...,...,...,...
17,user_16540390.md,2023-08,0.167442
18,user_16578424.md,2023-08,0.166346
19,user_16764520.md,2023-08,0.163726
20,user_16895625.md,2023-08,0.186149


In [60]:
loaded_objects = loaded_objects.rename(columns = {"User1": "owner_user_id"})
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('.md', ''))
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('user_', ''))
# Add T_d (Treated: 2022-09 ~ 2023-08 (12 months) Cntrol: 2021-09 ~ 2022-08 (12 months) )
import numpy as np
T_d = np.zeros(len(loaded_objects), dtype = int)
T_d[264:] = 1
loaded_objects['T_d'] = T_d
# Make month variable
loaded_objects['month'] = loaded_objects['year_month'].str[5:].astype(int).astype(str)
# Add P_t (if  9, 10, 11 month == P_t == 0, the others P_t == 1)
loaded_objects['P_t'] = loaded_objects['month'].apply(lambda x: 0 if x in ['9', '10', '11'] else 1)
loaded_objects = loaded_objects.rename(columns={'Similarity_toAvg': 'LSM'})
loaded_objects['ln_LSM'] = np.log(loaded_objects['LSM'])
loaded_objects

In [72]:
df_analysis_new['year_month'] = df_analysis_new['year_month'].astype(str)
df_final_new = pd.merge(loaded_objects, df_analysis_new[['owner_user_id', 'year_month', 'community']], on = ["owner_user_id", 'year_month'], how = 'left')
df_final_new

,owner_user_id,year_month,LSM,T_d,month,P_t,ln_LSM,community
0,14460824,2021-09,0.170243,0,9,0,-1.770528,17
1,14536215,2021-09,0.162675,0,9,0,-1.816002,17
2,14624409,2021-09,0.175510,0,9,0,-1.740057,17
3,14672114,2021-09,0.162087,0,9,0,-1.819622,4
4,14704799,2021-09,0.171497,0,9,0,-1.763192,7
...,...,...,...,...,...,...,...,...
523,16540390,2023-08,0.167442,1,8,1,-1.787120,0
524,16578424,2023-08,0.166346,1,8,1,-1.793683,8
525,16764520,2023-08,0.163726,1,8,1,-1.809561,4
526,16895625,2023-08,0.186149,1,8,1,-1.681206,7


In [73]:
# Aggregate Monthly (24*15 = 360)
df_cluster_new = df_final_new.groupby(['year_month', 'community'])['LSM'].mean().reset_index()
df_cluster_new

,year_month,community,LSM
0,2021-09,0,0.174137
1,2021-09,4,0.167801
2,2021-09,6,0.168966
3,2021-09,7,0.178315
4,2021-09,8,0.168589
...,...,...,...
163,2023-08,6,0.212015
164,2023-08,7,0.236795
165,2023-08,8,0.164431
166,2023-08,13,0.189239


In [76]:
# Add T_d
T_d = np.zeros(len(df_cluster_new), dtype = int)
T_d[84:] = 1
df_cluster_new['T_d'] = T_d
# Add P_t
# Make month variable
df_cluster_new['month'] = df_cluster_new['year_month'].str[5:].astype(int).astype(str)
# Add P_t (if  9, 10, 11 month == P_t == 0, the others P_t == 1)
df_cluster_new['P_t'] = df_cluster_new['month'].apply(lambda x: 0 if x in ['9', '10', '11'] else 1)

In [77]:
df_cluster_new

,year_month,community,LSM,T_d,month,P_t
0,2021-09,0,0.174137,0,9,0
1,2021-09,4,0.167801,0,9,0
2,2021-09,6,0.168966,0,9,0
3,2021-09,7,0.178315,0,9,0
4,2021-09,8,0.168589,0,9,0
...,...,...,...,...,...,...
163,2023-08,6,0.212015,1,8,1
164,2023-08,7,0.236795,1,8,1
165,2023-08,8,0.164431,1,8,1
166,2023-08,13,0.189239,1,8,1


In [82]:
# Get techiness
df_final_new = pd.read_csv("/data1/StackOverflow/_Robustness/H4/DDD/df_final_new.csv")
techiness = df_final_new.groupby('community')['techiness'].mean().reset_index()
df_cluster_new = pd.merge(df_cluster_new, techiness, on = 'community', how = 'left')
df_cluster_new

,year_month,community,LSM,T_d,month,P_t,techiness
0,2021-09,0,0.174137,0,9,0,0.525040
1,2021-09,4,0.167801,0,9,0,0.439161
2,2021-09,6,0.168966,0,9,0,0.414720
3,2021-09,7,0.178315,0,9,0,0.338053
4,2021-09,8,0.168589,0,9,0,0.304784
...,...,...,...,...,...,...,...
163,2023-08,6,0.212015,1,8,1,0.414720
164,2023-08,7,0.236795,1,8,1,0.338053
165,2023-08,8,0.164431,1,8,1,0.304784
166,2023-08,13,0.189239,1,8,1,0.421145


In [83]:
# ln(LSM)
df_cluster_new['ln_LSM'] = np.log(df_cluster_new['LSM'])
df_cluster_new = df_cluster_new[['year_month', 'month', 'community', 'techiness', 'T_d', 'P_t', 'LSM', 'ln_LSM']]
df_cluster_new

,year_month,month,community,techiness,T_d,P_t,LSM,ln_LSM
0,2021-09,9,0,0.525040,0,0,0.174137,-1.747915
1,2021-09,9,4,0.439161,0,0,0.167801,-1.784977
2,2021-09,9,6,0.414720,0,0,0.168966,-1.778058
3,2021-09,9,7,0.338053,0,0,0.178315,-1.724206
4,2021-09,9,8,0.304784,0,0,0.168589,-1.780290
...,...,...,...,...,...,...,...,...
163,2023-08,8,6,0.414720,1,1,0.212015,-1.551098
164,2023-08,8,7,0.338053,1,1,0.236795,-1.440559
165,2023-08,8,8,0.304784,1,1,0.164431,-1.805267
166,2023-08,8,13,0.421145,1,1,0.189239,-1.664744


In [84]:
# df_final_pre4.csv has the 'monthly' numUser value calculated by each cluster.
import pandas as pd
data = pd.read_csv('/data1/StackOverflow/_Robustness/H4/DDD/df_final_new_vr2.csv')
numUser = data.groupby(['year_month','community'])['numUser'].mean().reset_index()
numUser['numUser'] = numUser['numUser'].astype(int)

import numpy as np
df_cluster_new = pd.merge(df_cluster_new, numUser, on = ['year_month', 'community'], how = 'left')
df_cluster_new['ln_numUser'] = np.log(df_cluster_new['numUser'])
df_cluster_new

,year_month,month,community,techiness,T_d,P_t,LSM,ln_LSM,numUser,ln_numUser
0,2021-09,9,0,0.525040,0,0,0.174137,-1.747915,8865,9.089866
1,2021-09,9,4,0.439161,0,0,0.167801,-1.784977,3118,8.044947
2,2021-09,9,6,0.414720,0,0,0.168966,-1.778058,3589,8.185629
3,2021-09,9,7,0.338053,0,0,0.178315,-1.724206,1173,7.067320
4,2021-09,9,8,0.304784,0,0,0.168589,-1.780290,990,6.897705
...,...,...,...,...,...,...,...,...,...,...
163,2023-08,8,6,0.414720,1,1,0.212015,-1.551098,3422,8.137980
164,2023-08,8,7,0.338053,1,1,0.236795,-1.440559,1901,7.550135
165,2023-08,8,8,0.304784,1,1,0.164431,-1.805267,1345,7.204149
166,2023-08,8,13,0.421145,1,1,0.189239,-1.664744,237,5.468060


In [86]:
df_cluster_new.describe()

,community,techiness,T_d,P_t,LSM,ln_LSM,numUser,ln_numUser
count,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000
mean,7.857143,0.418041,0.500000,0.750000,0.180735,-1.714667,4758.851190,7.885237
std,5.237229,0.071337,0.501495,0.434307,0.016869,0.087139,4432.491015,1.233222
min,0.000000,0.304784,0.000000,0.000000,0.159365,-1.836556,189.000000,5.241747
25%,4.000000,0.338053,0.000000,0.750000,0.169019,-1.777745,1268.000000,7.145195
50%,7.000000,0.421145,0.500000,1.000000,0.175019,-1.742863,3421.500000,8.137834
75%,13.000000,0.483382,1.000000,1.000000,0.188992,-1.666051,9233.000000,9.130526
max,17.000000,0.525040,1.000000,1.000000,0.276427,-1.285809,16990.000000,9.740380


In [87]:
df_cluster_new.to_csv("lsm_cluster_new.csv", index = False)

#### 2-2-2 Oldbie

In [2]:
import pickle
with open('df_analysis_old.pickle', 'rb') as fw:
    df_analysis_old = pickle.load(fw)
print("Number of observation: ", len(df_analysis_old))
print("NumUser x Num year_months: ", df_analysis_old.owner_user_id.nunique()*df_analysis_old.year_month.nunique())

Number of observation:  18432
NumUser x Num year_months:  18432


In [3]:
# LSM Score Aggregation
import os
import pandas as pd
folder_path = '/data1/StackOverflow/_Robustness/H4/DDD/lsm/lsm_result/oldbie'
# Get a list of all files in the folder
file_list = os.listdir(folder_path)
file_list = sorted(file_list)
# Filter only pickle files
pickle_files = [file for file in file_list if file.endswith('.pickle')]

# Iterate through each pickle file and load it into a separate object
loaded_objects = pd.DataFrame(columns = ['User1', 'year_month', 'Similarity_toAvg'])
for pickle_file in pickle_files:
    file_path = os.path.join(folder_path, pickle_file)
    file_name = os.path.splitext(pickle_file)[0]
    # AGGREGATION
    with open(file_path, 'rb') as f:
        loaded_object = pickle.load(f)
    loaded_object['Similarity_toAvg'] = loaded_object['Similarity_toAvg'].astype(float)
    loaded_object = loaded_object.groupby('User1')['Similarity_toAvg'].mean().reset_index()
    loaded_object['year_month'] = file_name # Add year_month
    loaded_object = loaded_object[['User1', 'year_month', 'Similarity_toAvg']]
    # APPEND
    loaded_objects = pd.concat([loaded_objects, loaded_object])

/tmp/ipykernel_1429840/1195612365.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  loaded_objects = pd.concat([loaded_objects, loaded_object])


In [7]:
loaded_objects = loaded_objects.rename(columns = {"User1": "owner_user_id"})
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('.md', ''))
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('user_', ''))
# Add T_d (Treated: 2022-09 ~ 2023-08 (12 months) Cntrol: 2021-09 ~ 2022-08 (12 months) )
import numpy as np
T_d = np.zeros(len(loaded_objects), dtype = int)

###############
T_d[9204:] = 1 # Change this
###############

loaded_objects['T_d'] = T_d
# Make month variable
loaded_objects['month'] = loaded_objects['year_month'].str[5:].astype(int).astype(str)
# Add P_t (if  9, 10, 11 month == P_t == 0, the others P_t == 1)
loaded_objects['P_t'] = loaded_objects['month'].apply(lambda x: 0 if x in ['9', '10', '11'] else 1)
loaded_objects = loaded_objects.rename(columns={'Similarity_toAvg': 'LSM'})
loaded_objects['ln_LSM'] = np.log(loaded_objects['LSM'])
loaded_objects

,owner_user_id,year_month,LSM,T_d,month,P_t,ln_LSM
0,10008173,2021-09,0.180098,0,9,0,-1.714252
1,10112124,2021-09,0.189457,0,9,0,-1.663594
2,10138734,2021-09,0.176145,0,9,0,-1.736450
3,10140124,2021-09,0.170091,0,9,0,-1.771421
4,10157127,2021-09,0.180466,0,9,0,-1.712210
...,...,...,...,...,...,...,...
762,9952196,2023-08,0.170452,1,8,1,-1.769300
763,9957710,2023-08,0.167969,1,8,1,-1.783978
764,99692,2023-08,0.172772,1,8,1,-1.755782
765,9971759,2023-08,0.170607,1,8,1,-1.768393


In [8]:
df_analysis_old['year_month'] = df_analysis_old['year_month'].astype(str)
df_final_old = pd.merge(loaded_objects, df_analysis_old[['owner_user_id', 'year_month', 'community']], on = ["owner_user_id", 'year_month'], how = 'left')
# Aggregate Monthly (24*15 = 360)
df_cluster_old = df_final_old.groupby(['year_month', 'community'])['LSM'].mean().reset_index()
df_cluster_old

,year_month,community,LSM
0,2021-09,0,0.187851
1,2021-09,1,0.185182
2,2021-09,3,0.183932
3,2021-09,4,0.182470
4,2021-09,5,0.187970
...,...,...,...
355,2023-08,11,0.178409
356,2023-08,13,0.198197
357,2023-08,14,0.182675
358,2023-08,15,0.187097


In [11]:
# Add T_d (CHANGE THIS)
T_d = np.zeros(len(df_cluster_old), dtype = int)
T_d[180:] = 1
df_cluster_old['T_d'] = T_d
# Add P_t
# Make month variable
df_cluster_old['month'] = df_cluster_old['year_month'].str[5:].astype(int).astype(str)
# Add P_t (if  9, 10, 11 month == P_t == 0, the others P_t == 1)
df_cluster_old['P_t'] = df_cluster_old['month'].apply(lambda x: 0 if x in ['9', '10', '11'] else 1)

In [12]:
# Get techiness
df_final_old = pd.read_csv("/data1/StackOverflow/_Robustness/H4/DDD/df_final_old.csv")
techiness = df_final_old.groupby('community')['techiness'].mean().reset_index()
df_cluster_old = pd.merge(df_cluster_old, techiness, on = 'community', how = 'left')
# ln(LSM)
df_cluster_old['ln_LSM'] = np.log(df_cluster_old['LSM'])
df_cluster_old = df_cluster_old[['year_month', 'month', 'community', 'techiness', 'T_d', 'P_t', 'LSM', 'ln_LSM']]
df_cluster_old

,year_month,month,community,techiness,T_d,P_t,LSM,ln_LSM
0,2021-09,9,0,0.525040,0,0,0.187851,-1.672107
1,2021-09,9,1,0.412791,0,0,0.185182,-1.686417
2,2021-09,9,3,0.490224,0,0,0.183932,-1.693188
3,2021-09,9,4,0.439161,0,0,0.182470,-1.701171
4,2021-09,9,5,0.461350,0,0,0.187970,-1.671472
...,...,...,...,...,...,...,...,...
355,2023-08,8,11,0.476285,1,1,0.178409,-1.723677
356,2023-08,8,13,0.421145,1,1,0.198197,-1.618492
357,2023-08,8,14,0.403302,1,1,0.182675,-1.700044
358,2023-08,8,15,0.307775,1,1,0.187097,-1.676129


In [13]:
# df_final_pre4.csv has the 'monthly' numUser value calculated by each cluster.
import pandas as pd
data = pd.read_csv('/data1/StackOverflow/_Robustness/H4/DDD/df_final_old_vr2.csv')
numUser = data.groupby(['year_month','community'])['numUser'].mean().reset_index()
numUser['numUser'] = numUser['numUser'].astype(int)

import numpy as np
df_cluster_old = pd.merge(df_cluster_old, numUser, on = ['year_month', 'community'], how = 'left')
df_cluster_old['ln_numUser'] = np.log(df_cluster_old['numUser'])
df_cluster_old

,year_month,month,community,techiness,T_d,P_t,LSM,ln_LSM,numUser,ln_numUser
0,2021-09,9,0,0.525040,0,0,0.187851,-1.672107,13451,9.506809
1,2021-09,9,1,0.412791,0,0,0.185182,-1.686417,5246,8.565221
2,2021-09,9,3,0.490224,0,0,0.183932,-1.693188,4814,8.479284
3,2021-09,9,4,0.439161,0,0,0.182470,-1.701171,4602,8.434246
4,2021-09,9,5,0.461350,0,0,0.187970,-1.671472,4880,8.492900
...,...,...,...,...,...,...,...,...,...,...
355,2023-08,8,11,0.476285,1,1,0.178409,-1.723677,205,5.323010
356,2023-08,8,13,0.421145,1,1,0.198197,-1.618492,393,5.973810
357,2023-08,8,14,0.403302,1,1,0.182675,-1.700044,85,4.442651
358,2023-08,8,15,0.307775,1,1,0.187097,-1.676129,68,4.219508


In [14]:
df_cluster_old.describe()

,community,techiness,T_d,P_t,LSM,ln_LSM,numUser,ln_numUser
count,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000
mean,8.200000,0.426193,0.500000,0.750000,0.183293,-1.697414,3056.041667,7.017482
std,4.989571,0.070612,0.500696,0.433615,0.007247,0.038310,3236.373714,1.776816
min,0.000000,0.304784,0.000000,0.000000,0.169076,-1.777407,23.000000,3.135494
25%,4.000000,0.373225,0.000000,0.750000,0.178632,-1.722428,294.750000,5.686126
50%,8.000000,0.421145,0.500000,1.000000,0.183167,-1.697356,3077.500000,8.031853
75%,13.000000,0.483382,1.000000,1.000000,0.186497,-1.679339,4455.750000,8.401950
max,17.000000,0.541652,1.000000,1.000000,0.243471,-1.412757,13451.000000,9.506809


In [15]:
df_cluster_old.to_csv("lsm_cluster_old.csv", index = False)